![SemanticKernel](../assets/semantic-kernel.png)

# Microsoft Semantic Kernel Demo

Vamos a crear dos ejemplos empleando la librería de Semantic Kernel para entender el modelo de agentes.

## Dependencias y Referencias

Primero instalaremos las siguientes dependencias:

 - `dotenv.net` → es una biblioteca de .NET que facilita la gestión de variables de entorno en tus proyectos.
 - `Microsoft.SemanticKernel` → contiene las abstracciones base que impulsan el resto del ecosistema de Semantic Kernel. Estas abstracciones están diseñadas para ser modulares y simples, permitiendo que cualquier proveedor implemente la interfaz requerida y se integre fácilmente con el resto del ecosistema. Ofrece variantes por defecto para la gran mayoría de acciones.
 - `Microsoft.SemanticKernel.Agents.Core` → contiene las abstracciones básicas de Semantic Kernel para la creación de agentes y asistentes.
 - `Microsoft.SemanticKernel.Agents.OpenAI` → contiene las abstraciones específicas para emplear modelos de OpenAI en la creación de agentes y asistentes.

In [1]:
// Install the DotNetEnv package using the NuGet package manager
#r "nuget: dotenv.net, 3.2.1"
#r "nuget: Microsoft.SemanticKernel, 1.30.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.30.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.30.0-alpha"

Installed Packages dotenv.net, 3.2.1 Microsoft.SemanticKernel, 1.30.0 Microsoft.SemanticKernel.Agents.Core, 1.30.0-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.30.0-alpha

Con estas dependencias instaladas, importamos los siguienbtes recursos:

- `dotenv.net` → Importa la capacidad de leer las variables de entorno desde un archivo `.env`. La llamada `DotEnv.Load();` lee el archivo `.env` y carga las variables definidas en él como variables de entorno del sistema. Esto es útil para gestionar configuraciones sensibles como claves API y credenciales sin hardcodearlas en el código fuente.
- `Microsoft.SemanticKernel` → Importa las clases y abstracciones base de Semantic Kernel.
- `Microsoft.SemanticKernel.Connectors.OpenAI` → Importa las integraciones para OpenAI a través de su SDK, tanto de la propia empresa detrás de ChatGPT como la versión de Azure OpenAI (o cualquier otra).
- `Microsoft.SemanticKernel.Agents` → Librería experimental que importa las clases y abstracciones necesarias para la capacidad de agentes en Semantic Kernel.
- `Microsoft.SemanticKernel.Agents.Chat` → Librería experimental que importa las clases y abstracciones necesarias para la capacidad de chat entre agentes en Semantic Kernel.
- `Microsoft.SemanticKernel.Agents.History` → Librería experimental que importa las clases y abstracciones necesarias para la capacidad de histórico de conversaciones entre agentes en Semantic Kernel.

In [2]:
using System.ComponentModel;
using System.Text.Json;
using System.Threading;

using dotenv.net;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Libraries for agents from Semantic Kernel
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.History;
using Microsoft.SemanticKernel.Agents.OpenAI;

using ChatTokenUsage = OpenAI.Chat.ChatTokenUsage;

// Load the .env file
DotEnv.Load();

Finalmente leemos las variables de entorno que nos proporcionan los siguientes valores:

- `AZURE_OPENAI_API_KEY` → Es la clave secreta de autenticación y autorización para emplear la instancia del servicio de Azure OpenAI que tengamos desplegado en nuestra instancia de la nube de Microsoft.
- `AZURE_OPENAI_ENDPOINT` → Es la dirección (o URL) desde la cual podemos acceder al servicio de Azure OpenAI que tengamos desplegado en nuestra instancia de la nube de Microsoft.
- `AZURE_OPENAI_DEPLOYMENT_NAME` → Es el nombre del despliegue del modelo (LLM) que vamos a utilizar.
- `AZURE_OPENAI_MODEL_NAME` → Es el nombre (o tipo) del modelo que vamos a utilizar. Por ejemplo `gpt-4o`.
- `AZURE_OPENAI_API_VERSION` → Es la versión del API de Azure con la cual estarmos interactuando con el servicio. Por ejemplo `2024-02-15-preview`.

In [3]:

// Access the environment variables
var azureOpenAIEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var azureOpenAIKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
var azureOpenAIModel = Environment.GetEnvironmentVariable("AZURE_OPENAI_MODEL_NAME"); 
var azureOpenAIDeployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT_NAME");
var azureOpenAIApiVersion = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_VERSION");

In [4]:
#pragma warning disable OPENAI001
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0110

private static readonly JsonSerializerOptions jsonOptionsCache = new() { WriteIndented = true };

public static string AsJson(this object obj)
{
    return JsonSerializer.Serialize(obj, jsonOptionsCache);
}

public Kernel CreateKernelWithChatCompletion()
{
    var builder = Kernel.CreateBuilder();

    builder.AddAzureOpenAIChatCompletion(azureOpenAIDeployment, azureOpenAIEndpoint, azureOpenAIKey);

    return builder.Build();
}

public void WriteAgentChatMessage(ChatMessageContent message)
{
    // Include ChatMessageContent.AuthorName in output, if present.
    string authorExpression = message.Role == AuthorRole.User ? string.Empty : $" - {message.AuthorName ?? "*"}";
    
    // Include TextContent (via ChatMessageContent.Content), if present.
    string contentExpression = string.IsNullOrWhiteSpace(message.Content) ? string.Empty : message.Content;
    bool isCode = message.Metadata?.ContainsKey(OpenAIAssistantAgent.CodeInterpreterMetadataKey) ?? false;
    string codeMarker = isCode ? "\n  [CODE]\n" : " ";
    
    Console.WriteLine($"\n# {message.Role}{authorExpression}:{codeMarker}{contentExpression}");

    if (message.Metadata?.TryGetValue("Usage", out object usage) ?? false)
    {
        if (usage is ChatTokenUsage chatUsage)
        {
            Console.WriteLine($"  [Usage] Tokens: {chatUsage.TotalTokenCount}, Input: {chatUsage.InputTokenCount}, Output: {chatUsage.OutputTokenCount}");
        }
    }
}

In [5]:
private const string CreativeDirectorName = "CreativeDirector";
private const string CreativeDirectorInstructions =
    """
    You are a creative director who has opinions about copywriting born of a love for David Ogilvy and Harrison McCann.
    Your goal is to determine if a given copy is acceptable to print, even if it isn't perfect. 
    If not, provide insight on how to refine suggested copy without example.
    Always respond to the most recent message by evaluating and providing critique without example.    
    If copy is acceptable and meets your criteria, state that it is approved.
    If not, provide insight on how to refine suggested copy without examples.
    """;

private const string CopywriterName = "Copywriter";
private const string CopywriterInstructions =
    """
    You are a copywriter with ten years of experience and are known for brevity and a dry humor.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Your goal is to refine and decide on one single best copy as an expert in the field.
    Just create one copy.
    Consider suggestions when refining an idea.
    """;

In [6]:
#pragma warning disable SKEXP0110

ChatCompletionAgent agentReviewer = new()
{
    Instructions = CreativeDirectorInstructions,
    Name = CreativeDirectorName,
    Kernel = CreateKernelWithChatCompletion(),
};

ChatCompletionAgent agentWriter = new()
{
    Instructions = CopywriterInstructions,
    Name = CopywriterName,
    Kernel = CreateKernelWithChatCompletion(),
};

In [7]:
#pragma warning disable SKEXP0110

KernelFunction terminationFunction = AgentGroupChat.CreatePromptFunctionForStrategy(
    """
    Determine if the copy has been approved.  If so, respond with a single word: yes

    History:
    {{$history}}
    """,
    safeParameterNames: "history");

KernelFunction selectionFunction = AgentGroupChat.CreatePromptFunctionForStrategy(
    $$$"""
    Determine which participant takes the next turn in a conversation based on the the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.
    
    Choose only from these participants:
    - {{{CreativeDirectorName}}}
    - {{{CopywriterName}}}
    
    Always follow these rules when selecting the next participant:
    - After {{{CopywriterName}}}, it is {{{CreativeDirectorName}}}'s turn.
    - After {{{CreativeDirectorName}}}, it is {{{CopywriterName}}}'s turn.

    History:
    {{$history}}
    """,
    safeParameterNames: "history");

In [8]:
#pragma warning disable SKEXP0110

// Limit history used for selection and termination to the most recent message.
ChatHistoryTruncationReducer strategyReducer = new(1);

In [9]:
#pragma warning disable SKEXP0110

// Create a chat for agent interaction.
AgentGroupChat chat = new(agentWriter, agentReviewer)
{
    ExecutionSettings =
        new()
        {
            // Here KernelFunctionTerminationStrategy will terminate
            // when the art-director has given their approval.
            TerminationStrategy = new KernelFunctionTerminationStrategy(terminationFunction, CreateKernelWithChatCompletion())
            {
                // Only the art-director may approve.
                Agents = [agentReviewer],
                // Customer result parser to determine if the response is "yes"
                ResultParser = (result) => result.GetValue<string>()?.Contains("yes", StringComparison.OrdinalIgnoreCase) ?? false,
                // The prompt variable name for the history argument.
                HistoryVariableName = "history",
                // Limit total number of turns
                MaximumIterations = 10,
                // Save tokens by not including the entire history in the prompt
                HistoryReducer = strategyReducer,
            },

            // Here a KernelFunctionSelectionStrategy selects agents based on a prompt function.
            SelectionStrategy = new KernelFunctionSelectionStrategy(selectionFunction, CreateKernelWithChatCompletion())
            {
                // Always start with the writer agent.
                InitialAgent = agentWriter,
                // Returns the entire result value as a string.
                ResultParser = (result) => result.GetValue<string>() ?? CopywriterName,
                // The prompt variable name for the history argument.
                HistoryVariableName = "history",
                // Save tokens by not including the entire history in the prompt
                HistoryReducer = strategyReducer,
            },
        }
};

// Invoke chat and display messages.
ChatMessageContent message = new(AuthorRole.User, "concept: maps made out of egg cartons.");
chat.AddChatMessage(message);
WriteAgentChatMessage(message);

await foreach (ChatMessageContent responese in chat.InvokeAsync())
{
    WriteAgentChatMessage(responese);
}

Console.WriteLine($"\n[IS COMPLETED: {chat.IsComplete}]");


# user: concept: maps made out of egg cartons.

# Assistant - Copywriter: "Discover the world. One egg carton at a time."
  [Usage] Tokens: 103, Input: 91, Output: 12

# Assistant - CreativeDirector: The copy "Discover the world. One egg carton at a time." holds the potential for piquing curiosity and imagination with its concise and engaging sentence. However, to strengthen its effectiveness further, consider emphasizing the uniqueness of transforming something mundane into a vivid exploration tool. This might help the audience to better visualize and appreciate the innovation behind the concept.
  [Usage] Tokens: 216, Input: 147, Output: 69

# Assistant - Copywriter: "Turn breakfast waste into world-class geography."
  [Usage] Tokens: 195, Input: 186, Output: 9

# Assistant - CreativeDirector: The phrase "Turn breakfast waste into world-class geography" presents an intriguing concept, drawing attention to the transformation of reusable materials. However, the term "waste" might carr

In [10]:
#pragma warning disable SKEXP0001

using System.Diagnostics;
using System.Net.Http;

using Microsoft.SemanticKernel.TextToImage;

internal sealed class TimePlugin
{
    [KernelFunction]
    [Description("Retrieves the current time in UTC")]
    public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
}

internal sealed class ImagePlugin
{
    private readonly IChatCompletionService chatCompletionService;
    private readonly ITextToImageService textToImageService;

    public ImagePlugin(Kernel kernel)
    {
        this.chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();
        this.textToImageService = kernel.GetRequiredService<ITextToImageService>();
    }

    [KernelFunction]
    [Description("Creates an image from a text description")]
    public async Task<string> CreateImageFromTextAsync(string description, CancellationToken cancellationToken)
    {
        var imageTask = textToImageService.GenerateImageAsync(description, 1024, 1024, cancellationToken: cancellationToken);

        var systemPrompt = $@"Create a human response to indicate users that the image they requested has been created. The prompt for the image the user has requested is: {description}";

        var messageTask = chatCompletionService.GetChatMessageContentAsync(systemPrompt, new OpenAIPromptExecutionSettings()
        {
            MaxTokens = 50,
            Temperature = 1.0,
            TopP = 1.0,
        }, cancellationToken: cancellationToken);

        await Task.WhenAll(imageTask, messageTask);

        return $"{messageTask.Result.Content!} \n\n URL: {imageTask.Result}";
    }
}

internal sealed class WeatherPlugin
{
    private readonly IChatCompletionService chatCompletionService;
    private readonly HttpClient httpClient;

    private readonly string weatherStackKey;

    public WeatherPlugin(Kernel kernel)
    {
        this.chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();
        this.httpClient = new HttpClient();
        
        weatherStackKey = Environment.GetEnvironmentVariable("WEATHER_STACK_KEY");
    }

    [KernelFunction]
    [Description("Gets the current weather for the specified city")]
    public async Task<string> GetWeatherForCityAsync(string cityName, CancellationToken cancellationToken)
    {
        using var request = new HttpRequestMessage(HttpMethod.Get, $@"https://api.weatherstack.com/current?query={cityName}&access_key={weatherStackKey}");
        using var response = await httpClient.SendAsync(request, cancellationToken);
        response.EnsureSuccessStatusCode();

        var content = await response.Content.ReadAsStringAsync(cancellationToken);

        var systemPrompt = $$"""
            You are an expert AI understanding and interpreting the JSON response from the Weatherstack service. Create a easily to read and short summary of the weather from the following JSON:
            
            {{content}}
            
            """;

        var result = await chatCompletionService.GetChatMessageContentAsync(systemPrompt, new OpenAIPromptExecutionSettings()
        {
            MaxTokens = 200,
            Temperature = 0.1,
            TopP = 1.0,
        }, cancellationToken: cancellationToken);

        return result.Content!;
    }
}

static void ShowKernelResults(ChatMessageContent result, ChatHistory plan, int elapsedSeconds)
{
    ShowPlanFromChatHistory(plan);

    Console.ForegroundColor = ConsoleColor.Yellow;
    Console.WriteLine("\nKernel execution result:");
    Console.ForegroundColor = ConsoleColor.Cyan;
    Console.WriteLine(result);

    Console.ForegroundColor = ConsoleColor.Yellow;
    Console.WriteLine($"\nKernel execution total time: {elapsedSeconds} seconds");

    Console.ResetColor();
}

static void ShowPlanFromChatHistory(ChatHistory chatHistory)
{
    Console.ForegroundColor = ConsoleColor.Yellow;
    Console.WriteLine("The plan is:\n");
    foreach (var item in chatHistory)
    {
        ConsoleColor roleColor;

        if (item.Role == AuthorRole.System)
        {
            roleColor = ConsoleColor.DarkBlue;
        }
        else if (item.Role == AuthorRole.Tool)
        {
            roleColor = ConsoleColor.DarkMagenta;
        }
        else if (item.Role == AuthorRole.User)
        {
            roleColor = ConsoleColor.DarkGreen;
        }
        else if (item.Role == AuthorRole.Assistant)
        {
            roleColor = ConsoleColor.DarkYellow;
        }
        else
        {
            roleColor = ConsoleColor.DarkGray;
        }

        Console.BackgroundColor = roleColor;
        Console.ForegroundColor = ConsoleColor.White;
        Console.Write($"{item.Role.Label}");

        Console.ResetColor();
        Console.ForegroundColor = roleColor;
        Console.WriteLine($"\n{item.Content!}\n");

        Console.ResetColor();
    }
}

In [11]:
#pragma warning disable SKEXP0010

var azureOpenAIImageModel = Environment.GetEnvironmentVariable("AZURE_OPENAI_IMAGE_MODEL_NAME"); 
var azureOpenAIImageDeployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_IMAGE_DEPLOYMENT_NAME");

public Kernel CreateKernelWithChatCompletionAndTextToImage()
{
    var builder = Kernel.CreateBuilder();

    builder.AddAzureOpenAIChatCompletion(azureOpenAIDeployment, azureOpenAIEndpoint, azureOpenAIKey)
           .AddAzureOpenAITextToImage(azureOpenAIImageDeployment, azureOpenAIEndpoint, azureOpenAIKey, azureOpenAIImageModel);

    return builder.Build();
}

In [13]:
var cancellationTokenSource = new CancellationTokenSource();
var kernel = CreateKernelWithChatCompletionAndTextToImage();
var imagePlugin = new ImagePlugin(kernel);
var timePlugin = new TimePlugin();
var weatherPlugin = new WeatherPlugin(kernel);

kernel.ImportPluginFromObject(imagePlugin);
kernel.ImportPluginFromObject(timePlugin);
kernel.ImportPluginFromObject(weatherPlugin);

const string Goal = @"Check current UTC time, then tell me the current weather in Madrid city, and finally use that information from the weather to create an image.";

ChatHistory chatHistory = new();
chatHistory.AddUserMessage(Goal);

OpenAIPromptExecutionSettings executionSettings = new() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };

var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

var stopwatch = Stopwatch.StartNew();

var result = await chatCompletionService.GetChatMessageContentAsync(chatHistory, executionSettings, kernel, cancellationTokenSource.Token);

stopwatch.Stop();

ShowKernelResults(result, chatHistory, stopwatch.Elapsed.Seconds);

The plan is:

user
Check current UTC time, then tell me the current weather in Madrid city, and finally use that information from the weather to create an image.

Assistant


tool
Wed, 27 Nov 2024 16:58:54 GMT

Assistant


tool
The current weather in Madrid, Spain, is sunny with a temperature of 13°C, though it feels like 14°C. The wind is coming from the south-southeast at a gentle speed of 4 km/h. The atmospheric pressure is 1025 mb, and there is no precipitation. Humidity is at 47%, with clear skies and no cloud cover. Visibility is excellent at 10 km, and the UV index is low at 0. The local time is 5:45 PM.

Assistant


tool
Your image has been successfully created! ☀️ Enjoy a bright and sunny day in Madrid, depicted with clear skies and a gentle breeze. The temperature is set at a refreshing 13°C, all under the radiant sun with perfect visibility. Feel free 

 URL: https://dalleprodsec.blob.core.windows.net/private/images/b6b47af2-2265-4dfe-ae83-f338f510a4cb/generated_00.png?se=20